In [1]:
import pandas as pd
import numpy as np

In [2]:
# Crosswalk from place to county
# data link: http://mcdc.missouri.edu/cgi-bin/uexplore?/data/corrlst

# PROBLEM: Nashville is a (balance) instead of city. Bethesda is a CDP instead of city. 
# Check for more cases like this and somehow fix

xwalk = pd.read_csv('data/place_county.csv')
# get cities and state 2dig abbrv
xwalk = xwalk.loc[xwalk['PlaceName'].str.endswith('city')]
xwalk['city'] = xwalk['PlaceName'].str.split(' city').str[0]
xwalk['state'] = xwalk['CntyName'].str[-2:]
xwalk['county'] = xwalk['county'].astype(int)
xwalk['place_2000_pop'] = xwalk['pop100']
# just keep necessary columns
xwalk = xwalk.loc[:, ['state', 'county','city', 'afact', 'place_2000_pop']]
xwalk.head(5)

,state,county,city,afact,place_2000_pop
1,AL,1067,Abbeville,1.000,2987
2,AL,1073,Adamsville,1.000,4965
5,AL,1117,Alabaster,1.000,22619
6,AL,1095,Albertville,1.000,17247
7,AL,1123,Alexander City,1.000,15008


In [3]:
# FCC data via Tedi-- thank you!
fccraw = pd.read_stata('data/US_2018_county.dta')
## frn codes don't map perfectly to provider name, but it just seems to be a renaming issue
# x = (fcc.groupby('frn')['providername'].unique().str.len() > 1)
# ntrue = x[x == 1]
# fcc[fcc['frn'] == 4056248]

## get number of providers by city, weighting by 2000 county population
# number of providers per county
fcc = fccraw.groupby('county')['frn'].nunique().reset_index()
fcc = fcc.merge(xwalk, on = 'county')

# check merge
fcc[fcc['county'] == 47037]

fcc['afact'] = fcc['afact'].astype(float)

# weighted average function
def wavg(df):
    return((df['frn'] * df['afact']).sum().round())

# weighted average number of providers by city
fcc_gby = fcc.groupby(['state','city'])
fcc = fcc_gby.apply(wavg).reset_index()
fcc = fcc.rename(columns = {0: 'w_avg_numproviders'})
fcc = fcc.merge(fcc_gby['frn'].mean().reset_index()
                .rename(columns = {'frn': 'avg_numproviders'}), on = ['state', 'city'])
fcc = fcc.merge(fcc_gby['frn'].median().reset_index()
                .rename(columns = {'frn': 'median_numproviders'}), on = ['state', 'city'])

fcc.head(5)

,state,city,w_avg_numproviders,avg_numproviders,median_numproviders
0,AK,Akhiok,3.0,3.0,3.0
1,AK,Akiak,3.0,3.0,3.0
2,AK,Akutan,3.0,3.0,3.0
3,AK,Aleknagik,4.0,4.0,4.0
4,AK,Allakaket,7.0,7.0,7.0


In [4]:
# Work from home data
work = pd.read_csv('data/horton_data/gcs-apr5_w_states.csv')
# city is the 4th geographic identifier in the "geo" column
work['city'] = work['geo'].str.split('-').str[3]

# count number of observations with city data-- 13599/25K. 54% of sample
print("Number of obs with city data: {}".format(work['city'].notnull().sum()))

# count number of cities matched to fcc data
merged = work.merge(fcc, on = ['state', 'city'], how = 'left')
print("Number of matched obs: {}".format(merged['avg_numproviders'].notnull().sum()))

# unmatched cities:
print('Unmerged GEOs')
print(merged[(merged['avg_numproviders'].isnull()) & (merged['city'].notnull())]['geo'].unique())

merged.head(5)


Number of obs with city data: 13599
Number of matched obs: 11233
Unmerged GEOs
['US-SOUTH-TN-Nashville' 'US-WEST-HI-Napili-Honokowai'
 'US-NORTHEAST-NJ-Lakewood Township' 'US-SOUTH-VA-Tuckahoe'
 'US-SOUTH-GA-Rex' 'US-WEST-ID-Boise' 'US-NORTHEAST-NY-Queensbury'
 'US-NORTHEAST-PA-Huntingdon' 'US-MIDWEST-IN-Schererville'
 'US-WEST-AZ-Marana' 'US-SOUTH-SC-Hilton Head Island'
 'US-MIDWEST-WI-Plover' 'US-SOUTH-NC-Winston-Salem' 'US-SOUTH-AL-Shelby'
 'US-WEST-CA-Goleta' 'US-SOUTH-LA-Sun' 'US-MIDWEST-MO-KCMO'
 'US-SOUTH-DE-Pike Creek' 'US-WEST-AK-Anchorage'
 'US-NORTHEAST-RI-Westerly' 'US-NORTHEAST-NY-Ballston'
 'US-SOUTH-VA-Reston' 'US-WEST-HI-Wailuku' 'US-MIDWEST-IN-Indianapolis'
 'US-MIDWEST-MI-Cassopolis' 'US-MIDWEST-MO-Affton' 'US-WEST-NV-Kingsbury'
 'US-SOUTH-DE-Glasgow' 'US-MIDWEST-WI-Caledonia'
 'US-SOUTH-TN-Collierville' 'US-WEST-AZ-Sahuarita'
 'US-SOUTH-MD-Washington' 'US-NORTHEAST-ME-Falmouth'
 'US-SOUTH-VA-Abingdon' 'US-MIDWEST-WI-Mount Pleasant'
 'US-WEST-NV-Incline Village' 'US-W

,Unnamed: 0,id,time,status,pub_cat,gender,age,geo,weight,q,response_time,region,state,city,w_avg_numproviders,avg_numproviders,median_numproviders
0,1,100012282,2020-04-02 14:00:00,Complete,News,Unknown,Unknown,US-SOUTH-FL-Ocala,0.000000,None of the above / Not working for pay,9272,SOUTH,FL,Ocala,22.0,22.0,22.0
1,2,1000296325,2020-04-02 03:00:00,Complete,News,Male,55-64,US-SOUTH-AL-Muscle Shoals,0.868838,I continue to commute to work,22582,SOUTH,AL,Muscle Shoals,12.0,12.0,12.0
2,3,1000475325,2020-04-01 22:00:00,Complete,Reference,Female,55-64,US-MIDWEST-MI,0.626525,None of the above / Not working for pay,5775,MIDWEST,MI,NaN,NaN,NaN,NaN
3,4,1000492508,2020-04-02 15:00:00,Complete,News,Male,55-64,US-SOUTH-TN-Nashville,0.868838,I continue to commute to work,20659,SOUTH,TN,Nashville,NaN,NaN,NaN
4,5,1000512424,2020-04-03 14:00:00,Complete,News,Female,45-54,US-SOUTH-TX,0.966719,Used to work from home and still do,32449,SOUTH,TX,NaN,NaN,NaN,NaN


In [5]:
## get number working numbers by city

# indicator for type of survey respondent
qs = list(merged['q'].unique())
for q in merged['q'].unique():
    merged[q] = merged['q'] == q
    
# create final city-level dataset: number of providers, number of respondents not working, working remotely, etc. 
mgby = merged.groupby(['state', 'city'])
final = mgby[['avg_numproviders', 'median_numproviders']].max().reset_index()
final = final.merge(
    mgby[qs].sum().reset_index(),
    on = ['state','city'])

# merge population data
final = final.merge(xwalk[['state', 'city', 'place_2000_pop']], on = ['state', 'city'], how = 'left')

final.to_csv('data/city_broadband_work.csv')
final.head(5)

,state,city,avg_numproviders,median_numproviders,None of the above / Not working for pay,I continue to commute to work,Used to work from home and still do,"Used to commute, now work from home",I have recently been furloughed or laid-off,"Used to work from home, but now I commute",place_2000_pop
0,AK,Anchorage,NaN,NaN,12.0,7.0,1.0,6.0,1.0,0.0,NaN
1,AK,Wasilla,5.0,5.0,5.0,0.0,0.0,2.0,1.0,0.0,5469
2,AL,Alabaster,19.0,19.0,9.0,6.0,2.0,2.0,1.0,0.0,22619
3,AL,Andalusia,9.0,9.0,3.0,1.0,0.0,1.0,0.0,0.0,8794
4,AL,Atmore,14.0,14.0,3.0,2.0,0.0,1.0,0.0,0.0,7676


In [6]:
check = pd.read_csv('data/place_county.csv')
# check[check['PlaceName'].str.startswith('Nashville')]
# check[check['PlaceName'].str.startswith('Bethesda')]
check

,state,placefp,county,msacmsa,pop100,afact,PlaceName,CntyName,MetroName,Primary
0,,,County (FIPS),Metro Area (MSA or CMSA) 2000,Complete Count Pop 2k Census,Portion of place pop in County,,,,Primary County Flag
1,01,00124,01067,Non-metro,2987,1.000,Abbeville city,Henry AL,NaN,1
2,01,00460,01073,"Birmingham, AL",4965,1.000,Adamsville city,Jefferson AL,1000,1
3,01,00484,01133,Non-metro,723,1.000,Addison town,Winston AL,NaN,1
4,01,00676,01065,Non-metro,521,1.000,Akron town,Hale AL,NaN,1
...,...,...,...,...,...,...,...,...,...,...
26297,56,84852,56001,Non-metro,100,1.000,Woods Landing-Jelm CDP,Albany WY,NaN,1
26298,56,84925,56043,Non-metro,5250,1.000,Worland city,Washakie WY,NaN,1
26299,56,85015,56005,Non-metro,1347,1.000,Wright town,Campbell WY,NaN,1
26300,56,86665,56015,Non-metro,169,1.000,Yoder town,Goshen WY,NaN,1
